In [21]:
import torch
import sys
import os
import rasterio

parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)

from src.ccsds import create_lossless_compressor, decompress
from tests.utils import generate_simple_test_image

# Create test image [Z, Y, X]
#image = torch.randn(10, 64, 64) * 100  # 10 bands, 64x64 pixels
batch_size = 1
num_bands = 5
dynamic_range = 16
#images = [generate_simple_test_image(num_bands=num_bands, dynamic_range=dynamic_range) for _ in range(batch_size)]
#batch = torch.stack(images)
image = generate_simple_test_image(num_bands=num_bands, dynamic_range=dynamic_range)
print(f'Image shape: {image.shape}\n')
print(f'Image Bit: {image.dtype}\n')

Image shape: torch.Size([5, 16, 16])

Image Bit: torch.float32



In [49]:
import rasterio
tif_path = '/home/joshua/Documents/phd_university/code/HSI code examples/aviris_hyperspectral_data/19920612_AVIRIS_IndianPine_Site3.tif'

# Open the TIFF file
with rasterio.open(tif_path) as src:
    # Read the entire file into a numpy array
    hsi_data = src.read()

hsi = torch.from_numpy(hsi_data).float()
hsi.shape

torch.Size([220, 145, 145])

In [ ]:
# Create lossless compressor
compressor = create_lossless_compressor(num_bands=num_bands, dynamic_range=dynamic_range)

In [ ]:
# Compress to get complete compressed representation
compressed_data = compressor(image)
print(f"Compression ratio: {compressed_data['compression_statistics']['compression_ratio']:.2f}:1")
print(f"Compressed size: {len(compressed_data['compressed_bitstream'])} bytes")

In [ ]:
# Decompress to reconstruct image
reconstructed = decompress(compressed_data)
print(f"Lossless: max error = {torch.max(torch.abs(image - reconstructed))}")

In [50]:
# For strict CCSDS-123.0-B-2 compliance
from src.optimized import create_optimized_lossless_compressor

optimized_compressor = create_optimized_lossless_compressor(
    num_bands=hsi.shape[0],
    optimization_mode='causal',  # Maintains exact causal order
    dynamic_range=dynamic_range,
)

optimized_compressor.device

device(type='cuda')

In [52]:
results = optimized_compressor(hsi)

In [53]:
results.keys()

dict_keys(['predictions', 'residuals', 'quantized_residuals', 'mapped_indices', 'sample_representatives', 'reconstructed_samples', 'compressed_size', 'original_size', 'compression_ratio', 'compression_time', 'throughput_samples_per_sec', 'entropy_stats'])

In [54]:
print(results['compressed_size'])
print(results['compression_ratio'])
print(results['mapped_indices'].shape)
for key, stat in results['entropy_stats'].items():
    print(f'{key}: {stat}')

129360600
0.572106189983658
torch.Size([220, 145, 145])
total_samples: 4625500
total_compressed_bits: 129360600
total_compressed_bytes: 16170075
bits_per_sample: 27.966836017727815
compression_ratio: 0.572106189983658
encoding_time: 103.01276421546936
throughput_samples_per_sec: 44902.20251079712
band_stats: [{'band': 0, 'total_samples': 21025, 'high_entropy_samples': 21016, 'low_entropy_samples': 9, 'high_entropy_ratio': 0.9995719381688466, 'compressed_bits': 588395, 'encoding_time': 0.363008975982666, 'bits_per_sample': 27.98549346016647}, {'band': 1, 'total_samples': 21025, 'high_entropy_samples': 21016, 'low_entropy_samples': 9, 'high_entropy_ratio': 0.9995719381688466, 'compressed_bits': 588333, 'encoding_time': 0.3524658679962158, 'bits_per_sample': 27.98254458977408}, {'band': 2, 'total_samples': 21025, 'high_entropy_samples': 21019, 'low_entropy_samples': 6, 'high_entropy_ratio': 0.9997146254458977, 'compressed_bits': 588314, 'encoding_time': 0.35024070739746094, 'bits_per_samp

In [57]:
print(results['reconstructed_samples'].shape)

torch.Size([220, 145, 145])


In [55]:
from src.ccsds import calculate_psnr

In [58]:
print(calculate_psnr(hsi.to('cpu'), results['reconstructed_samples'].to('cpu'), 16))

218.3483731891304


In [ ]:
compressed_data = optimized_compressor.compress(hsi)